In [126]:
import pandas as pd
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

In [127]:
derecho_calls=pd.read_csv("../data/911_Phone Calls_Derecho_050320 050420.csv")

In [128]:
tornado_calls=pd.read_csv("../data/911_Phone_Calls_Tornado_030320.csv")

In [129]:
derecho_comp=pd.read_csv("../data/Computer Aided Dispatch Data_Derecho Incidents 050320-050420.csv", skiprows=3)

In [130]:
tornado_comp=pd.read_csv("../data/Computer Aided Dispatch Data_Tornado Incidents 030320.csv", skiprows=3)

In [131]:
tornado_calls.columns = ['datetime', 'calltype','lat','lng','address']

In [132]:
derecho_comp.columns = ['datetime', 'calltype','lat','lng','alarm']

In [133]:
tornado_comp.columns = ['datetime', 'calltype','lat','lng','alarm']

In [134]:
derecho_calls.columns = ['datetime', 'calltype','lat','lng','address']

#### Timeline of each storm/ Beginning and end of the event
Tornado calls: 3/3/2020 12:34am-4:58am

Tornado computer: 12:36am-4:57am

Derecho calls: 5/3/2020 4:30pm- 5/4/2020 12:29am

Derecho computer: 4:31pm-12:29am

#### Stonger storm impact based no call volumes
Tornado Calls:
    Datetime - 
            12:43am= 12 calls           
            12:58am and 12:59am = 11 calls         
    Address - 
        Wireless caller                     101     
        907 12th av n-n                     16     
        1051G Chicamauga Ave - S            10     
        3333 Lebanon Pike - SE              8       
        4940 Old Hickory BLVD. - SE SECTOR  7
    
Tornado Comp: 
    Datetime- 3/3/2020 
            12:47:35 AM   2
            1:28:24 AM    2
            1:15:14 AM    2
            12:44:02 AM   2
            2:04:00 AM    2
    Alarm-
            GENERAL ALARM COMMERCIAL   INDUSTRIAL BUILD      27
            OUTSIDE ODOR                                     26
            WIRES DOWN WITH NO SMOKE OR ARCING               17
            SMALL BUILDING INVESTIGATION                     15
            ELECTRICAL HAZARD WITH OR NEAR WATER             12
Derecho Calls:
     Datetime- 
             5/3/2020 18:58    12
             5/3/2020 16:36    12
             5/3/2020 18:48    11
             5/3/2020 18:04    11
             5/3/2020 17:19    10
     Address-
            WIRELESS CALLER                339
            4400 HARDING PIKE - E           15
            3333 LEBANON PIKE - N           14
            WIRELESS 911 CALLER             14
            3009D DARRINGTON WAY - N        14    
Derecho Comp:
      Datetime-
             5/3/2020 7:48:02 PM     2
             5/3/2020 7:16:05 PM     2
             5/3/2020 9:39:26 PM     2
             5/3/2020 6:35:07 PM     2
             5/3/2020 9:29:25 PM     2
       Alarm-
           ELECTRICAL HAZARD                                370
           GENERAL ALARM COMMERCIAL   INDUSTRIAL BUILD       23
           MVA INJURIES:                                      8
           SMALL BUILDING INVESTIGATION                       6
           SMALL OUTSIDE FIRE                                 6  

In [143]:
derecho_comp['alarm'].value_counts()

ELECTRICAL HAZARD                                370
GENERAL ALARM COMMERCIAL   INDUSTRIAL BUILD       23
MVA INJURIES:                                      8
SMALL BUILDING INVESTIGATION                       6
SMALL OUTSIDE FIRE                                 6
                                                ... 
REQUEST ADDITIONAL FIRE RESOURCES                  1
SEIZURE: EFFECTIVE BREATHING NOT VERIFIED >35      1
NOT ALERT ASTHMA                                   1
SUICIDAL  NOT THREATENING  ALERT                   1
OBVIOUS DEATH                                      1
Name: alarm, Length: 122, dtype: int64

In [86]:
derecho_comp['geometry'] = derecho_comp.apply(lambda x: Point((float(x.lng), 
                                                         float(x.lat))), 
                                        axis=1)

In [87]:
tornado_calls['geometry'] = tornado_calls.apply(lambda x: Point((float(x.lng), 
                                                         float(x.lat))), 
                                        axis=1)

In [88]:
tornado_comp['geometry'] = tornado_comp.apply(lambda x: Point((float(x.lng), 
                                                         float(x.lat))), 
                                        axis=1)

In [124]:
derecho_calls['geometry'] = derecho_calls.apply(lambda x: Point((float(x.lng), 
                                                         float(x.lat))), 
                                        axis=1)
derecho_comp.head(10)

,datetime,calltype,lat,lng,address,geometry
0,5/3/2020 4:31:07 PM,901 KENWICK CT W,36.046877,-86.962526,ELECTRICAL HAZARD,POINT (-86.96253 36.04688)
1,5/3/2020 4:32:18 PM,8036 ARBOR DR,36.082491,-86.940680,ELECTRICAL HAZARD,POINT (-86.94068 36.08249)
2,5/3/2020 4:32:23 PM,119 DUE WEST AV E,36.248029,-86.716875,NOT ALERT COPD,POINT (-86.71687 36.24803)
3,5/3/2020 4:33:04 PM,630 GALLATIN PKE S,36.256491,-86.714817,ABDOMINAL PAIN,POINT (-86.71482 36.25649)
4,5/3/2020 4:33:52 PM,615 W HILLWOOD DR,36.129690,-86.879834,ELECTRICAL HAZARD,POINT (-86.87983 36.12969)
5,5/3/2020 4:33:53 PM,622 BROOK HOLLOW RD,36.111773,-86.888087,ELECTRICAL HAZARD,POINT (-86.88809 36.11177)
6,5/3/2020 4:34:19 PM,JOCELYN HOLLOW RD / BRESSLYN RD,36.103318,-86.898080,ELECTRICAL HAZARD,POINT (-86.89808 36.10332)
7,5/3/2020 4:35:01 PM,WOODLAND ST / S 7TH ST,36.173583,-86.760339,SEIZURE: EFFECTIVE BREATHING NOT VERIFIED >35,POINT (-86.76034 36.17358)
8,5/3/2020 4:36:42 PM,3000 POSTON AV,36.145332,-86.815356,GENERAL ALARM COMMERCIAL INDUSTRIAL BUILD,POINT (-86.81536 36.14533)
9,5/3/2020 4:37:04 PM,51ST AV N / CENTENNIAL BLVD,36.162326,-86.849144,ELECTRICAL HAZARD,POINT (-86.84914 36.16233)


In [91]:
zipcodes = gpd.read_file('../data/zipcodes.geojson')
print(zipcodes.crs)
zipcodes.head(2)

epsg:4326


,zip,objectid,po_name,shape_stlength,shape_starea,geometry
0,37115,1,MADISON,178783.02488886821,596553400.57885742,"MULTIPOLYGON (((-86.68725 36.31821, -86.68722 ..."
1,37216,3,NASHVILLE,75820.997821400058,188884682.28344727,"MULTIPOLYGON (((-86.73451 36.23774, -86.73425 ..."


In [92]:
zipcodes.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [95]:
tornado_calls_geo = gpd.GeoDataFrame(tornado_calls, 
                           crs = zipcodes.crs, 
                           geometry = tornado_calls['geometry'])
type(tornado_calls_geo)

geopandas.geodataframe.GeoDataFrame

In [96]:
derecho_calls_geo = gpd.GeoDataFrame(derecho_calls, 
                           crs = zipcodes.crs, 
                           geometry = derecho_calls['geometry'])
type(derecho_calls_geo)

geopandas.geodataframe.GeoDataFrame

In [97]:
tornado_comp_geo = gpd.GeoDataFrame(tornado_comp, 
                           crs = zipcodes.crs, 
                           geometry = tornado_comp['geometry'])
type(tornado_comp_geo)

geopandas.geodataframe.GeoDataFrame

In [98]:
derecho_comp_geo = gpd.GeoDataFrame(derecho_comp, 
                           crs = zipcodes.crs, 
                           geometry = derecho_comp['geometry'])
type(derecho_comp_geo)

geopandas.geodataframe.GeoDataFrame

In [99]:
zipcodes = zipcodes[['zip', 'po_name', 'geometry']]

In [100]:
tornado_calls_zip = gpd.sjoin(tornado_calls_geo, zipcodes, op = 'within')

In [101]:
tornado_comp_zip = gpd.sjoin(tornado_comp_geo, zipcodes, op = 'within')

In [102]:
derecho_calls_zip = gpd.sjoin(derecho_calls_geo, zipcodes, op = 'within')

In [103]:
derecho_comp_zip = gpd.sjoin(derecho_comp_geo, zipcodes, op = 'within')

In [107]:
nashville = (36.16784, -86.77816)

nash_map = folium.Map(location = nashville, zoom_start = 10)
nash_map

In [110]:
new_map = folium.Map(location =  nashville, zoom_start = 14)

#create a marker cluster
marker_cluster = MarkerCluster().add_to(new_map)

folium.GeoJson(zipcodes).add_to(marker_cluster)

# inside the loop add each markwe to the cluster
for row_index, row_values in tornado_calls_zip.iterrows():
    loc = [row_values['lat'], row_values['lng']]
    pop = str(row_values['address']),row_values['calltype']
    icon=folium.Icon(color="red",icon="phone", prefix='fa')
    
    marker = folium.Marker(
        location = loc, 
        popup = pop,
    icon = icon) 
    
    marker.add_to(marker_cluster)

    
#save an interactive HTML map by calling .save()
new_map.save('../maps/tornado_nash.html')

new_map  